In [ ]:
import pandas as pd
import urllib
import sqlalchemy as sa
import json
import altair as alt
import folium


params = urllib.parse.quote_plus("DRIVER={SQL Server};"
                                 "SERVER=sql-kbenp.database.windows.net;"
                                 "DATABASE=provgron-staging;"
                                 "UID=<Vraag deze op>;"
                                 "PWD=<Vraag deze op>;")


engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params), fast_executemany=True, encoding='utf-8')

query ="""
SELECT top (100) [deviceId]
,      SUBSTRING([Start_date],1,10)                              as [Date]
,      substring([Start_date],12,5) as [Hour]
,      [Device_streetname]
,      (SUM([Number_of_objects_counted_in])  +   SUM([Number_of_objects_counted_out]))  as [Number_of_objects_counted]
,      [Type_of_device]
,      [latitude]
,      [longitude]
,      [latitude_measurement]
,      [longitude_measurement]
,      [latitude_direction_fw_count]
,      [longitude_direction_fw_count]
,      [latitude_direction_bw_count]
,      [longitude_measurement_bw_count]
FROM [dbo].[Passantaantallen]



Where Convert(date,SUBSTRING([Start_date],1,10)) >= '2020-06-26'
	AND Convert(date,SUBSTRING([Start_date],1,10)) <= '2020-06-30'

GROUP BY [deviceId]
,        SUBSTRING([Start_date],1,10)
,		     SUBSTRING([Start_date],12,5)
,        [Device_streetname]
,        [Type_of_device]
,        [latitude]
,        [longitude]
,        [latitude_measurement]
,        [longitude_measurement]
,        [latitude_direction_fw_count]
,        [longitude_direction_fw_count]
,        [latitude_direction_bw_count]
,        [longitude_measurement_bw_count]

order by  [Date] desc,[Hour] desc

"""

# Pandas dataframe (of your choice)
data = pd.read_sql_query(query, con=engine, index_col=None)

data['Tijd'] = pd.to_datetime(data['Date'] + ' ' + data['Hour'])

data = data.rename(columns={
    'Number_of_objects_counted': 'Aantal passanten',
    'latitude_measurement': 'lat',
    'longitude_measurement': 'lon',
    'Device_streetname': 'Straatnaam'
})

In [205]:
def make_subplot(plot_data: pd.DataFrame, width: int=560, height: int=320):
    """
    Make a bar plot. Edit this function to make a plot of your choice. https://altair-viz.github.io/gallery/index.html#simple-charts
    :return: altair chart object 
    """
    
    chart = alt.Chart(
        plot_data
    ).mark_line(
        interpolate='basis'
    ).encode(
        x='Tijd',
        y='Aantal passanten'
    ).properties(
        width=width,
        height=height
    )
    
    return chart

In [206]:
# Get camera in each street
cameras = data.groupby('Straatnaam').first().index.tolist()

# Initialise a folium map with a pair of coordinates in the data
coordinaten = data[['lat', 'lon']].values.tolist()
map_with_tiles = folium.Map(location=coordinaten[0], tiles='CartoDB positron')

for camera in cameras:
    # Make individual dataframe for each camera 
    plot_data = data.loc[data['Straatnaam']==camera].sort_values('Tijd')
    
    # Get the coordingates for this camera
    coordinates = plot_data[['lat', 'lon']].values.tolist()[0]
    
    popup = folium.Popup(max_width=640)
    
    # VegaLite object to store a plot
    folium.VegaLite(
        # altair plot in json format
        make_subplot(plot_data).to_json(), 
        height=420, 
        width=640
    ).add_to(popup)
    
    # Show this Vega object(plot) with a marker
    folium.Marker(
        coordinates, 
        popup=popup, 
        tooltip=f'{camera}'
    ).add_to(map_with_tiles)
    

In [207]:
map_with_tiles.save(outfile='test_map.html')